### 1.С помощью библиотеки numpy реализовать модель с прямым проходом, состоящую из 3 полносвязных слоёв с функциями потерь: ReLU, tanh, Softmax. Длины векторов на входе 256, на выходе 4, промежуточные: 64 и 16.

In [268]:
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

In [40]:
# инициализация исходных данных, весов на каждом слое
input_size, h1_size, h2_size, out_size = 256, 64, 16, 4

x = np.random.randn(input_size)

W1 = np.random.randn(input_size, h1_size)
W2 = np.random.randn(h1_size, h2_size)
W3 = np.random.randn(h2_size, out_size)

In [41]:
# функции активации
def relu(t):
    return np.maximum(t,0)

def tanh(t):
    return (np.exp(t)-np.exp(-t))/(np.exp(t)+np.exp(-t))

def softmax(t):
    out = np.exp(t)
    return out/np.sum(out)

In [42]:
# 1st layer
t1 = np.dot(x,W1)
h1 = relu(t1)
print("Result 1 layer: ", '\n', h1)

Result 1 layer:  
 [ 1.50495465 29.57903569  0.          0.8962552   0.          0.
  0.          0.         16.32254145  0.          0.          0.
 27.95105774  7.34052491  0.          0.          2.35886862  0.
  0.          0.          7.70470132 13.15876874  0.07028834 16.45157248
  6.97630032  7.88615722  0.         12.20056912  0.          0.
  0.         26.02841913  0.          0.         21.60925088 47.09567231
 19.90967021  6.51623539 26.70156257  0.          5.47276818  0.
  0.          0.          0.          0.          0.          3.02997184
  4.30634915  0.          0.          8.98628522  0.          0.51009178
  7.56609695  0.          0.          4.16987472 42.5150136   0.
 11.77882989 30.96572585 21.74034048 26.29836041]


In [43]:
# 2nd layer
t2 = np.dot(h1,W2)
h2 = tanh(t2)
print("Result 2 layer: ",'\n', h2)

Result 2 layer:  
 [-1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1. -1. -1.  1.]


In [25]:
# 3rd layer
t3 = np.dot(h2,W3)
h3 = softmax(t3)
print("Result 3 layer: ", h3)

Result 3 layer:  [0.0015802  0.87196204 0.09855957 0.0278982 ]


### 2. Реализовать модель с прямым проходом, состоящую из 2 свёрток с функциями активации ReLU и 2 функций MaxPool. Первый слой переводит из 3 каналов в 8, второй из 8 слоёв в 16. На вход подаётся изображение размера 19х19. (19х19x3 -> 18x18x8 -> 9x9x8 -> 8x8x16 -> 4x4x16)

In [249]:
# слои CNN
def convolution(image, filt):
    flt_amounts, flt_d, flt_h, flt_w = filt.shape 
    image_d, image_h, image_w = image.shape
    conv = np.zeros((flt_amounts,image_h-flt_h+1, image_h-flt_h+1)) 
    for flt_num in range(flt_amounts):
        curr_y = out_y = 0
        while curr_y + flt_h <= image_h:
            curr_x, out_x = 0, 0
            while curr_x + flt_h <= image_h:
                batch = image[:,curr_y:curr_y+flt_h, curr_x:curr_x+flt_h]
                conv[flt_num, out_y, out_x] = np.sum(filt[flt_num] * batch)
                curr_x += 1
                out_x += 1
            curr_y += 1
            out_y += 1
    return conv

def relu(t):
    return np.maximum(t,0)

def maxpool(image):
    image_d, image_h, image_w = image.shape
    pool = np.zeros((image_d, int((image_h - 2)/2)+1, int((image_w - 2)/2)+1)) 
    for i in range(image_d):
        curr_y = out_y = 0
        while curr_y + 2 <= image_h:
            curr_x, out_x = 0, 0
            while curr_x + 2 <= image_w:
                pool[i, out_y, out_x] = np.max(image[i, curr_y:curr_y+2, curr_x:curr_x+2])
                curr_x += 2
                out_x += 1
            curr_y += 2
            out_y += 1
    return pool

In [259]:
# инициализация изображения, фильтров и проход по слоям

image_h, image_w, image_d = 19, 19, 3
filter_h, filter_w, filter_d, fltr_amount = 2, 2, 3, 8
input_image = np.random.randint(0, 256, (image_d, image_h, image_w), dtype=np.uint8)
filt = np.random.randn(fltr_amount, filter_d, filter_h, filter_w)
filt1 = np.random.randn(16, 8, 2, 2)

conv1 = convolution(input_image,filt)
print("Размер после первой свертки: ", conv1.shape[::-1])
conv1_relu = relu(conv1)
max_pool1 = maxpool(conv1_relu)
print("Размер после первого пулинга: ", max_pool1.shape[::-1])

conv2 = convolution(max_pool1,filt1)
print("Размер после второй свертки: ", conv2.shape[::-1])
conv2_relu = relu(conv2)
max_pool2 = maxpool(conv2_relu)
print("Размер после второго пулинга: ", max_pool2.shape[::-1])


Размер после первой свертки:  (18, 18, 8)
Размер после первого пулинга:  (9, 9, 8)
Размер после второй свертки:  (8, 8, 16)
Размер после второго пулинга:  (4, 4, 16)


### 3. Объединить сети из п.2 и п.1. На вход изображение размера 19х19, на выходе вектор из 4 элементов

In [269]:
# инициализация изображения, фильтров и проход по слоям ( сеть 2 )

image_h, image_w, image_d = 19, 19, 3
print("Размер исходного изображения: ", conv1.shape[::-1])
filter_h, filter_w, filter_d, fltr_amount = 2, 2, 3, 8
input_image = np.random.randint(0, 256, (image_d, image_h, image_w), dtype=np.uint8)
filt = np.random.randn(fltr_amount, filter_d, filter_h, filter_w)
filt1 = np.random.randn(16, 8, 2, 2)

conv1 = convolution(input_image,filt)
print("Размер после первой свертки: ", conv1.shape[::-1])
conv1_relu = relu(conv1)
max_pool1 = maxpool(conv1_relu)
print("Размер после первого пулинга: ", max_pool1.shape[::-1])

conv2 = convolution(max_pool1,filt1)
print("Размер после второй свертки: ", conv2.shape[::-1])
conv2_relu = relu(conv2)
max_pool2 = maxpool(conv2_relu)
print("Размер после второго пулинга: ", max_pool2.shape[::-1])

# представляем изображение (4, 4, 16) вектором из 256 элементов
img_to_vector = np.ravel(max_pool2)
print("Размер после перевода в вектор: ", img_to_vector.shape)

# инициализируем веса и прогоняем через модель из 1 сети
input_size, h1_size, h2_size, out_size = 256, 64, 16, 4

W1 = np.random.randn(input_size, h1_size)
W2 = np.random.randn(h1_size, h2_size)
W3 = np.random.randn(h2_size, out_size)

t1 = np.dot(img_to_vector,W1)
h1 = relu(t1)
print("Размер после 1 слоя: ", h1.shape)

t2 = np.dot(h1,W2)
h2 = tanh(t2)
print("Размер после 2 слоя: ", h2.shape)

t3 = np.dot(h2,W3)
h3 = softmax(t3)
print("Размер после 3 слоя (ИТОГ): ", h3.shape)

Размер исходного изображения:  (18, 18, 8)
Размер после первой свертки:  (18, 18, 8)
Размер после первого пулинга:  (9, 9, 8)
Размер после второй свертки:  (8, 8, 16)
Размер после второго пулинга:  (4, 4, 16)
Размер после перевода в вектор:  (256,)
Размер после 1 слоя:  (64,)
Размер после 2 слоя:  (16,)
Размер после 3 слоя (ИТОГ):  (4,)
